In [ ]:
!pip install tensorflow
!pip install ffmpeg-python
!pip install einops
!pip install matplotlib
!pip install seaborn
!pip install keras
!pip install pydot-ng
!pip install scikit-learn

In [15]:
import tensorflow as tf
from tensorflow import keras

from sklearn import preprocessing as pp

import tqdm
import random
import pathlib
import itertools
import collections

import os
import cv2
import einops
import numpy as np
import remotezip as rz
import joblib

In [16]:
class Conv2Plus1D(keras.layers.Layer):
  def __init__(self, filters, kernel_size, padding):
    """
      A sequence of convolutional layers that first apply the convolution operation over the
      spatial dimensions, and then the temporal dimension. 
    """
    super().__init__()
    self.seq = keras.Sequential([  
        # Spatial decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(1, kernel_size[1], kernel_size[2]),
                      padding=padding),
        # Temporal decomposition
        layers.Conv3D(filters=filters, 
                      kernel_size=(kernel_size[0], 1, 1),
                      padding=padding)
        ])

  def get_config(self):
    config = super().get_config()
    #Update the config with the custom layer's parameters  
    return config
    
  def call(self, x):
    return self.seq(x)

In [17]:
class ResidualMain(keras.layers.Layer):
  """
    Residual block of the model with convolution, layer normalization, and the
    activation function, ReLU.
  """
  def __init__(self, filters, kernel_size):
    super().__init__()
    self.seq = keras.Sequential([
        Conv2Plus1D(filters=filters,
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization(),
        layers.ReLU(),
        Conv2Plus1D(filters=filters, 
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization()
    ])

  def get_config(self):
    config = super().get_config()
    #Update the config with the custom layer's parameters      
    return config
    
  def call(self, x):
    return self.seq(x)

In [18]:
class Project(keras.layers.Layer):
  """
    Project certain dimensions of the tensor as the data is passed through different 
    sized filters and downsampled. 
  """
  def __init__(self, units):
    super().__init__()
    self.seq = keras.Sequential([
        layers.Dense(units),
        layers.LayerNormalization()
    ])

  def get_config(self):
    config = super().get_config()
    return config
    
  def call(self, x):
    return self.seq(x)

In [19]:
def add_residual_block(input, filters, kernel_size):
  """
    Add residual blocks to the model. If the last dimensions of the input data
    and filter size does not match, project it such that last dimension matches.
  """
  out = ResidualMain(filters, 
                     kernel_size)(input)
  
  res = input
  # Using the Keras functional APIs, project the last dimension of the tensor to
  # match the new filter size
  if out.shape[-1] != input.shape[-1]:
    res = Project(out.shape[-1])(res)

  return layers.add([res, out])

In [20]:
class ResizeVideo(keras.layers.Layer):
  def __init__(self, height, width):
    super().__init__()
    self.height = height
    self.width = width
    self.resizing_layer = layers.Resizing(self.height, self.width)

  def get_config(self):
    config = super().get_config()
    return config

  def call(self, video):
    """
      Use the einops library to resize the tensor.  
      
      Args:
        video: Tensor representation of the video, in the form of a set of frames.
      
      Return:
        A downsampled size of the video according to the new height and width it should be resized to.
    """
    # b stands for batch size, t stands for time, h stands for height, 
    # w stands for width, and c stands for the number of channels.
    old_shape = einops.parse_shape(video, 'b t h w c')
    images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
    images = self.resizing_layer(images)
    videos = einops.rearrange(
        images, '(b t) h w c -> b t h w c',
        t = old_shape['t'])
    return videos

In [21]:
from tensorflow.keras.utils import get_custom_objects, register_keras_serializable
get_custom_objects().update({ "Conv2Plus1D": Conv2Plus1D })
register_keras_serializable(package= "MyLayers", name="Conv2Plus1D")(Conv2Plus1D)
get_custom_objects().update({ "ResidualMain": ResidualMain })
register_keras_serializable(package = "MyLayers", name = "ResidualMain")(ResidualMain)
get_custom_objects().update({ "ResizeVideo": ResizeVideo })
register_keras_serializable(package = "MyLayers", name = "ResizeVideo")(ResizeVideo)
get_custom_objects().update({ "Project": Project })
register_keras_serializable(package = "MyLayers", name = "Project")(Project)

__main__.Project

In [22]:
new_model = tf.keras.models.load_model('./complete_saved_model/test1.keras')
new_model.summary()

ValueError: File not found: filepath=./complete_saved_model/test1.keras. Please ensure the file is an accessible `.keras` zip file.